In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA silver")

In [0]:
# Renomeando as colunas para evitar caracteres não desejados

from functools import reduce

COLUMNS_TO_RENAME = {
    "ESTACAO": "station_code",
    "DATA (YYYY-MM-DD)": "date",
    "Hora UTC": "time",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "temperature",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)": "dew_point",
    "VENTO, VELOCIDADE HORARIA (m/s)": "wind_speed",
    "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "wind_direction",
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitation",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "pressure",
    "UMIDADE RELATIVA DO AR, HORARIA (%)": "relative_humidity",
    "VENTO, RAJADA MAXIMA (m/s)": "wind_gust",
    "RADIACAO GLOBAL (KJ/m²)": "radiation",
}

existing_columns = [
    col for col in COLUMNS_TO_RENAME.keys()
    if col in df_weather_data.columns
]

df_renamed_weather_data = reduce(
    lambda df, c: df.withColumnRenamed(c, COLUMNS_TO_RENAME[c]),
    existing_columns,
    df_weather_data.select(*existing_columns)
)

display(df_renamed_weather_data.limit(10))